In [1]:
from scipy.stats import linregress
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import json
#"key"=api_key
from config import api_key

In [ ]:
# url = "https://api.steampowered.com/ISteamUserStats/GetNumberOfCurrentPlayers/v1/"
# params = {"appid":'212910'}

# response = requests.get(url, params=params).json()
# print(json.dumps(response, indent=4, sort_keys=True))

In [ ]:
# appid_df = pd.DataFrame()
# appid_df['Player Count'] = ""


# #Google Search for top rated STEAM video games 
# ids = ['1174180', '715950', '499450', '590380', '593380', '231200', '1085660', '653530' ]
# url = "https://api.steampowered.com/ISteamUserStats/GetNumberOfCurrentPlayers/v1/"
# index = 0
# for appid in action_df.l:
#     params = {"appid":appid}
    
#     #rl = "https://api.steampowered.com/ISteamUserStats/GetNumberOfCurrentPlayers/v1/"
#     response = requests.get(url, params=params).json()
#     try:
#         player_count = response['response']['player_count']
    
#     except Exception as e:
        
#         print("did not find count") 
        
#     appid_df.loc[index,'Player Count'] = player_count
    
#     index = index + 1
# # response = requests.get(url, params=params).json()
# #print(json.dumps(player_count, indent=4, sort_keys=True))
# appid_df

In [ ]:
#Returns games with this particular tag
# url = 'https://steamspy.com/api.php?request=tag&tag=Early+Access'
# response = requests.get(url)
# response.json()

In [2]:
#Returns games in this particular genre. Requires *genre* parameter and works like this:
new_url = 'https://steamspy.com/api.php?request=genre&genre=action' #Early+Access'
response = requests.get(new_url).json()
response
action_df = pd.DataFrame(response).T
len(action_df)

24777

In [3]:
#Steps to create a clean data frame with all of the data from the action genre
#action_df = action_df.reset_index()
#del action_df['index']
#del action_df['level_0']
action_df.head()
#action_df.to_csv('Action_Genre_Data.csv', index=False)

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu
570,570,Dota 2,Valve,Valve,,1527142,315549,0,"200,000,000 .. 500,000,000",38515,1530,838,721,0,0,0,898826
730,730,Counter-Strike: Global Offensive,"Valve, Hidden Path Entertainment",Valve,,5930650,785631,0,"50,000,000 .. 100,000,000",29948,783,6352,259,0,0,0,869295
578080,578080,PUBG: BATTLEGROUNDS,"KRAFTON, Inc.","KRAFTON, Inc.",,1183129,907693,0,"50,000,000 .. 100,000,000",23073,710,7307,201,0,0,0,387793
1063730,1063730,New World,Amazon Games,Amazon Games,,161726,74387,0,"50,000,000 .. 100,000,000",8135,1184,3489,466,3999,3999,0,50197
440,440,Team Fortress 2,Valve,Valve,,844927,57340,0,"50,000,000 .. 100,000,000",9650,1335,366,183,0,0,0,97407


In [4]:
#Create a player count Table using the AppID
app_count = [] 
appid_list = []
#Used to test the for loop
# test_df = action_df.head()
#created to iterate through different portions of the action_df before connection to site errors out
set_df = action_df.iloc[17422:24777,0:2]

# search for player count by appid  
#url = "https://api.steampowered.com/ISteamUserStats/GetNumberOfCurrentPlayers/v1/"

for index, row in set_df.iterrows():
    appid = row['appid'] 
    name = row['name']
   
    print(f'checking index number {index} AppId number {appid} , Name {name} ')
    
    params = {'appid': appid}
    url = "https://api.steampowered.com/ISteamUserStats/GetNumberOfCurrentPlayers/v1/"
    
    response = requests.get(url, params=params).json()
    try:
        player_count = response['response']['player_count']
    
    except Exception as e:
        
        print("did not find count") 
        
    app_count.append(player_count)
    appid_list.append(appid)

checking index number 529100 AppId number 529100 , Name Omni Link 
checking index number 513920 AppId number 513920 , Name Desperate Times 
checking index number 530740 AppId number 530740 , Name Spareware 
checking index number 527330 AppId number 527330 , Name Grave: VR Prologue 
checking index number 529280 AppId number 529280 , Name TEOT - The End OF Tomorrow 
checking index number 511700 AppId number 511700 , Name Ship It 
checking index number 535530 AppId number 535530 , Name Demon Hunter 
checking index number 355230 AppId number 355230 , Name Faceted Flight 
checking index number 521860 AppId number 521860 , Name Lop Nor Zombie VR (HTC Vive) 
checking index number 522300 AppId number 522300 , Name Polywings 
checking index number 536530 AppId number 536530 , Name Marlene Betwixt 
checking index number 523720 AppId number 523720 , Name Unruly Ghouls 
checking index number 527300 AppId number 527300 , Name DOPAMINE 
checking index number 535010 AppId number 535010 , Name Pen Isl

checking index number 594150 AppId number 594150 , Name airRevo VR 
checking index number 603230 AppId number 603230 , Name The Wendigo 
checking index number 592240 AppId number 592240 , Name Jupiteration 
checking index number 597670 AppId number 597670 , Name Exterminator 
checking index number 395940 AppId number 395940 , Name Scribbled Arena 
checking index number 598240 AppId number 598240 , Name Stupid Raft Battle Simulator 
checking index number 548220 AppId number 548220 , Name Paladin 
checking index number 606030 AppId number 606030 , Name Meganoid 
checking index number 606360 AppId number 606360 , Name Mech Skeleton 
checking index number 601590 AppId number 601590 , Name BattleTrucks 
checking index number 599800 AppId number 599800 , Name Easy Magic 
checking index number 595990 AppId number 595990 , Name Princess Kidnapper 2 - VR 
checking index number 590570 AppId number 590570 , Name Paper Quest 
checking index number 585500 AppId number 585500 , Name Dick Wilde 
chec

checking index number 483490 AppId number 483490 , Name Circuitous  
checking index number 677700 AppId number 677700 , Name Attack of the Gooobers 
checking index number 673380 AppId number 673380 , Name SPECIAL FORCE VR 
checking index number 665670 AppId number 665670 , Name Enlysia 
checking index number 656330 AppId number 656330 , Name Outpost L5 
checking index number 635980 AppId number 635980 , Name Break The Food Chain 
checking index number 669570 AppId number 669570 , Name MicroSpy 
checking index number 673570 AppId number 673570 , Name Squadron: Sky Guardians 
checking index number 672690 AppId number 672690 , Name The Nothing 
checking index number 454750 AppId number 454750 , Name Aeternum 
checking index number 654240 AppId number 654240 , Name Xenoslaive Overdrive 
checking index number 666600 AppId number 666600 , Name Zombie Town 
checking index number 648070 AppId number 648070 , Name CrisisActionVR 
checking index number 651550 AppId number 651550 , Name ConcPerfe

checking index number 707770 AppId number 707770 , Name Tactics 2: War 
checking index number 691250 AppId number 691250 , Name Charlie the Duck 
checking index number 681950 AppId number 681950 , Name AddForce 
checking index number 722400 AppId number 722400 , Name Record Store Nightmare 
checking index number 686580 AppId number 686580 , Name Storms of Shambhala 
checking index number 721540 AppId number 721540 , Name Cursed Isles 
checking index number 619670 AppId number 619670 , Name Huge Bang Bang 
checking index number 701890 AppId number 701890 , Name Starzine 
checking index number 712430 AppId number 712430 , Name Vibrant 
checking index number 657020 AppId number 657020 , Name Save Dash 
checking index number 719110 AppId number 719110 , Name Imperil 
checking index number 624990 AppId number 624990 , Name The Mimic 
checking index number 723510 AppId number 723510 , Name The Cave VR 
checking index number 708140 AppId number 708140 , Name TriElement 
checking index number 

checking index number 732040 AppId number 732040 , Name Corridor 15 Firts 
checking index number 733180 AppId number 733180 , Name Destroy Space Aliens 
checking index number 731690 AppId number 731690 , Name Jingo 
checking index number 752770 AppId number 752770 , Name GyroShooter 
checking index number 718250 AppId number 718250 , Name Nova Flow 
checking index number 563550 AppId number 563550 , Name Nevrosa: Escape 
checking index number 654400 AppId number 654400 , Name Shank n' Bake 
checking index number 676310 AppId number 676310 , Name Deadly Zone 
checking index number 753960 AppId number 753960 , Name Jida Chronicle Chaos frontier VR 
checking index number 680400 AppId number 680400 , Name Light Tracer (VR & NON-VR) 
checking index number 713310 AppId number 713310 , Name Gambol 
checking index number 676580 AppId number 676580 , Name NEO Impossible Bosses 
checking index number 762120 AppId number 762120 , Name Nuke Babysitter Simulator | Kim Edition 
checking index number

...

checking index number 739890 AppId number 739890 , Name Dogstar 
checking index number 737660 AppId number 737660 , Name The Deepest House 
checking index number 746340 AppId number 746340 , Name HeadSquare - Multiplayer VR Ball Game 
checking index number 709620 AppId number 709620 , Name Squareboy vs Bullies: Arena Edition 
checking index number 749650 AppId number 749650 , Name Barrimean Jungle 
checking index number 705920 AppId number 705920 , Name Empress of Gold 
checking index number 751480 AppId number 751480 , Name Vector Velocity 
checking index number 687270 AppId number 687270 , Name Divide 
checking index number 556960 AppId number 556960 , Name Lost Wing 
checking index number 633080 AppId number 633080 , Name Relic Hunters Legend 
did not find count
checking index number 753390 AppId number 753390 , Name The Eerie Adventures Of Kally 
checking index number 733770 AppId number 733770 , Name MegaRace 3 
checking index number 746880 AppId number 746880 , Name Spoxel 
check

checking index number 751330 AppId number 751330 , Name Battery Jam 
checking index number 727970 AppId number 727970 , Name Cosmic Trail 
checking index number 748650 AppId number 748650 , Name Carnage in Space: Ignition 
checking index number 561110 AppId number 561110 , Name Disney's The Jungle Book 
checking index number 773930 AppId number 773930 , Name Coaster 
checking index number 768480 AppId number 768480 , Name Chamber of the Sci-Mutant Priestess 
checking index number 769390 AppId number 769390 , Name Xmas Zombie Rampage 2 
checking index number 767680 AppId number 767680 , Name Domino Effect 
checking index number 770660 AppId number 770660 , Name Thetaball 
checking index number 640250 AppId number 640250 , Name Kitty Hawk 
checking index number 773570 AppId number 773570 , Name Log Challenge 
checking index number 766500 AppId number 766500 , Name Dead Mist: Last Stand 
checking index number 768570 AppId number 768570 , Name Uncanny Islands 
checking index number 600180 

did not find count
checking index number 800710 AppId number 800710 , Name Mad Zombie 
checking index number 775570 AppId number 775570 , Name Deadly Blue 
checking index number 797190 AppId number 797190 , Name Survive: The king killer 
checking index number 777250 AppId number 777250 , Name The Tavern of Magic 
checking index number 611810 AppId number 611810 , Name THE LAST HUNT 
checking index number 795490 AppId number 795490 , Name FukTopia 
checking index number 724710 AppId number 724710 , Name Super Knockoff! VS 
checking index number 795440 AppId number 795440 , Name Why War? 
checking index number 802080 AppId number 802080 , Name Pillage 
checking index number 801010 AppId number 801010 , Name Unsolved Stories 
checking index number 790050 AppId number 790050 , Name Porcelain Panic 
checking index number 770270 AppId number 770270 , Name Warbot 
checking index number 783790 AppId number 783790 , Name Survivors of Borridor 
checking index number 787530 AppId number 787530 , 

checking index number 813280 AppId number 813280 , Name Sky Hawk 
checking index number 822770 AppId number 822770 , Name The Secrets of The Forest 
checking index number 818350 AppId number 818350 , Name TENKYU 
checking index number 807930 AppId number 807930 , Name War Of Spells 
checking index number 820690 AppId number 820690 , Name ZYTERNION 
checking index number 815860 AppId number 815860 , Name Reframed 
checking index number 816730 AppId number 816730 , Name HeadON! 
checking index number 765170 AppId number 765170 , Name CRAZY BIGHEADS 
checking index number 823290 AppId number 823290 , Name Pokka Man 
checking index number 679410 AppId number 679410 , Name The Adventures of Clive McMulligan on Planet Zeta Four 
checking index number 813930 AppId number 813930 , Name Revoke 
checking index number 820660 AppId number 820660 , Name Future City Coaster 
checking index number 821570 AppId number 821570 , Name Lethal Lawns: Competitive Mowing Bloodsport 
checking index number 788

ConnectionError: HTTPSConnectionPool(host='api.steampowered.com', port=443): Max retries exceeded with url: /ISteamUserStats/GetNumberOfCurrentPlayers/v1/?appid=846010 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000016392AAF288>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'))

In [5]:
#appid_df = pd.DataFrame()
#print(f'{appid_list} , {app_count}') 
len(appid_list)
len(app_count)

3743

In [6]:
#Create DF with appid and player count 
combineddata = {'appid':appid_list, 'player count':app_count}
combined_list_df = pd.DataFrame(combineddata)

combined_list_df

new_combined_df = pd.merge(combined_list_df,action_df, how="left", on=['appid','appid'])
new_combined_df.head()

,appid,player count,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu
0,529100,0,Omni Link,Roencia,Roencia Game Creators,,36,4,0,"0 .. 20,000",0,0,0,0,399,399,0,0
1,513920,0,Desperate Times,Yokcos,Yokcos,,8,2,0,"0 .. 20,000",0,0,0,0,499,499,0,0
2,530740,0,Spareware,Rusto,Rusto,,20,7,0,"0 .. 20,000",0,0,0,0,0,0,0,0
3,527330,0,Grave: VR Prologue,"Broken Window Studios, LLC","Broken Window Studios, LLC",,5,12,0,"0 .. 20,000",0,0,0,0,799,799,0,0
4,529280,0,TEOT - The End OF Tomorrow,"Edsenses Creative Co., Ltd","Edsenses Creative Co., Ltd",,9,9,0,"0 .. 20,000",0,0,0,0,399,399,0,0


In [7]:
updated_data_types = {'average_forever': float, 'average_2weeks': float, 'median_forever': float, 'negative':float, 'positive': float,
                      'median_2weeks': float,'price': float, 'initialprice': float, 'discount': float, 'ccu': float}

new_combined_df = new_combined_df.astype(updated_data_types)
new_combined_df.to_csv('AppID_PlayerCount_7.csv', index=False) 
new_combined_df

,appid,player count,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu
0,529100,0,Omni Link,Roencia,Roencia Game Creators,,36.0,4.0,0,"0 .. 20,000",0.0,0.0,0.0,0.0,399.0,399.0,0.0,0.0
1,513920,0,Desperate Times,Yokcos,Yokcos,,8.0,2.0,0,"0 .. 20,000",0.0,0.0,0.0,0.0,499.0,499.0,0.0,0.0
2,530740,0,Spareware,Rusto,Rusto,,20.0,7.0,0,"0 .. 20,000",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,527330,0,Grave: VR Prologue,"Broken Window Studios, LLC","Broken Window Studios, LLC",,5.0,12.0,0,"0 .. 20,000",0.0,0.0,0.0,0.0,799.0,799.0,0.0,0.0
4,529280,0,TEOT - The End OF Tomorrow,"Edsenses Creative Co., Ltd","Edsenses Creative Co., Ltd",,9.0,9.0,0,"0 .. 20,000",0.0,0.0,0.0,0.0,399.0,399.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3738,544380,0,Tesla: The Weather Man,"Thoughtquake Studios, LLC","Thoughtquake Studios, LLC",,6.0,1.0,0,"0 .. 20,000",0.0,0.0,0.0,0.0,299.0,299.0,0.0,0.0
3739,831520,0,Jurassic City Walk,Real Fighting,Real Fighting,,4.0,21.0,0,"0 .. 20,000",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3740,834340,0,The Hardest Dungeon,Pippala Games,Pippala Games,,16.0,4.0,0,"0 .. 20,000",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3741,818670,0,Save Snegurochka!,TIPA Games Berlin,TIPA Games Berlin,,5.0,0.0,0,"0 .. 20,000",0.0,0.0,0.0,0.0,899.0,899.0,0.0,0.0


In [17]:
#read in each of the csv files created
appid_playercount = pd.read_csv('./AppID_PlayerCount.csv')
appid_playercount2 = pd.read_csv('./AppID_PlayerCount_2.csv')
appid_playercount3 = pd.read_csv('./AppID_PlayerCount_3.csv')
appid_playercount4 = pd.read_csv('./AppID_PlayerCount_3.csv')
appid_playercount5 = pd.read_csv('./AppID_PlayerCount_5.csv')
appid_playercount6 = pd.read_csv('./AppID_PlayerCount_6.csv')
appid_playercount7 = pd.read_csv('./AppID_PlayerCount_7.csv')

In [19]:
# x_df = pd.concat(['appid_playercount','appid_playercount2', 'appid_playercount3', 'appid_playercount4appid_playercount4', 
#            'appid_playercount5', 'appid_playercount6', 'appid_playercount7']).to_csv('Fianl_Combined_Data.csv', index=False)

In [76]:
new_combined_df.sort_values('player count', ascending=False)
new_combined_df.head()

,appid,player count,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu
0,570,824806,Dota 2,Valve,Valve,,1525713.0,314885.0,0,"200,000,000 .. 500,000,000",39479.0,1624.0,805.0,841.0,0.0,0.0,0.0,898826.0
1,730,747220,Counter-Strike: Global Offensive,"Valve, Hidden Path Entertainment",Valve,,5928988.0,785454.0,0,"50,000,000 .. 100,000,000",30002.0,810.0,6578.0,251.0,0.0,0.0,0.0,869295.0
2,578080,187981,PUBG: BATTLEGROUNDS,"KRAFTON, Inc.","KRAFTON, Inc.",,1182862.0,907564.0,0,"50,000,000 .. 100,000,000",23130.0,766.0,7326.0,225.0,0.0,0.0,0.0,387793.0
3,1063730,51988,New World,Amazon Games,Amazon Games,,161642.0,74394.0,0,"50,000,000 .. 100,000,000",7949.0,816.0,3079.0,315.0,3999.0,3999.0,0.0,50197.0
4,440,112488,Team Fortress 2,Valve,Valve,,844706.0,57343.0,0,"50,000,000 .. 100,000,000",9778.0,940.0,353.0,167.0,0.0,0.0,0.0,97407.0


In [ ]:
# appid_info = []
# app_name = []
# avg_play_total_time = []
# avg_median_total = []
# avg_play_time_2weeks = []
# median_2Weeks = []
# price = []

In [ ]:
# #by App ID 
# list_ids = ['570','730', '578080','590380', '593380', '231200', '1085660', '653530', '1174180', '715950', '499450',]
# #new_url = 'https://steamspy.com/api.php?request=appdetails&appid='
          
# #params = {"appid":ids}
# for value in list_ids:
#     url= f'https://steamspy.com/api.php?request=appdetails&appid={value}'
   
#     response = requests.get(url).json()   
   
#     appid_info.append(response['appid'])
#     app_name.append(response['name'])
#     avg_play_total_time.append(response['average_forever'])
#     avg_median_total.append(response['median_forever'])
#     avg_play_time_2weeks.append(response['average_2weeks'])
#     median_2Weeks.append(response['median_2weeks'])
#     price.append(response['price'])

# print(app_name)   

#print(price)

In [ ]:

# steamdata = {'APP ID':appid_info,'App Name': app_name, 'Avg Total Play Time': avg_play_total_time,'Median Total Play Time': avg_median_total, 
#              'Avg Play Time 2 Weeks': avg_play_time_2weeks, 'Median 2 Weeks':median_2Weeks, 'Price':price
#             }
# # steam_df['APP ID'] = appid_info
# # steam_df['App Name'] = app_name
# # steam_df['Avg Play Time'] = avg_play_total_time
# # steam_df['Median Play Time'] =  avg_median_total
# # steam_df['Avg Play Time 2 Weeks'] = avg_play_time_2weeks
# # steam_df['Median 2 Weeks'] = median_2Weeks
# steam_df = pd.DataFrame(steamdata)
    
# steam_df

In [39]:
#Returns the Top 100 apps by players in the last two weeks
new_url = 'https://steamspy.com/api.php?request=top100in2weeks'
steam_df = pd.DataFrame()
steam_df['APP ID'] = ""
steam_df['App Name'] = ""
steam_df['Avg Play Time'] = ""
steam_df['Avg Play Time 2 Weeks'] = ""

# new_url = 'https://steamspy.com/api.php?request=top100in2weeks'
# params = { '?request=': 'top50', "length":"in2weeks"}
response = requests.get(new_url).json()
# response. weather_df.to_csv('WeatherPy.csv', index=False) 
top100_df = pd.DataFrame(response).T
top100_df
#top100_df.to_csv('Top100.csv', index=False) 

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu
1627140,1627140,Sabre Team,Krisalis Software,Ziggurat,,0,2,0,"1,000,000 .. 2,000,000",0,0,0,0,699,699,0,0
1619450,1619450,Heart of a Warrior,Techworld Communication,Techworld Communication,,0,1,0,"1,000,000 .. 2,000,000",0,0,0,0,1099,1099,0,0
1476720,1476720,Retro Run,Vander,lilleBror Media,,3,0,0,"500,000 .. 1,000,000",0,0,0,0,199,199,0,0
2063400,2063400,Descension,RedRock Game Studios,Brad Barber,,2,0,0,"500,000 .. 1,000,000",0,0,0,0,124,499,75,0
588140,588140,ESCAPE FROM VOYNA: Dead Forest,Dead Turtle,Dead Turtle,,0,1,0,"500,000 .. 1,000,000",0,0,0,0,699,699,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2103290,2103290,Anvil Life,Emre Bulut,Emre Bulut,,3,0,0,"50,000 .. 100,000",0,0,0,0,299,299,0,0
1886370,1886370,Labyrinth,SailingStudio,SailingStudio,,3,1,0,"50,000 .. 100,000",0,0,0,0,99,99,0,0
1726180,1726180,Old Gods,Poison Dart Frog,Poison Dart Frog,,1,0,0,"50,000 .. 100,000",0,0,0,0,599,599,0,0
42900,42900,Hearts of Iron III: Semper Fi,Paradox Development Studio,Paradox Interactive,,1,0,0,"50,000 .. 100,000",0,0,0,0,499,499,0,0


In [41]:
top100_df.dtypes

appid              object
name               object
developer          object
publisher          object
score_rank         object
positive           object
negative           object
userscore          object
owners             object
average_forever    object
average_2weeks     object
median_forever     object
median_2weeks      object
price              object
initialprice       object
discount           object
ccu                object
dtype: object

In [ ]:
#return apps with 1,000-1,999 owners.
new_url = 'https://steamspy.com/api.php?request=all&page=1'
steam_df = pd.DataFrame()
steam_df['APP ID'] = ""
steam_df['App Name'] = ""
steam_df['Avg Play Time'] = ""
steam_df['Avg Play Time 2 Weeks'] = ""

# new_url = 'https://steamspy.com/api.php?request=top100in2weeks'
# params = { '?request=': 'top50', "length":"in2weeks"}
response = requests.get(new_url).json()
# response. weather_df.to_csv('WeatherPy.csv', index=False) 
owners_df = pd.DataFrame(response).T
owners_df = owners_df.reset_index()
del owners_df['index']
owners_df
#response
owners_df.to_csv('SEAMS_Data.csv', index=False)
#weather_df.to_csv('WeatherPy.csv', index=False) 

In [ ]:
# new_url = 'https://steamspy.com/api.php?request=top100in2weeks'
# new_url2 = 'https://steamspy.com/api.php?request=appdetails&appid=231200'
# steam_df = pd.DataFrame()
# steam_df['APP ID'] = ""
# steam_df['App Name'] = ""
# steam_df['Avg Play Time'] = ""
# steam_df['Median Play Time'] = ""
# steam_df['Avg Play Time 2 Weeks'] = ""
# steam_df['Median Play Time 2 Weeks'] = ""
# # new_url = 'https://steamspy.com/api.php?request=top100in2weeks'
# # params = { '?request=': 'top50', "length":"in2weeks"}
# response = requests.get(new_url2).json()
# response 